In [46]:
import os
from collections import defaultdict, Counter
from text_utils import remove_reg, tokenise, get_provenance
import json
import numpy as np
from tqdm.notebook import tqdm
from wiki_labels import qid_lab_get

BASE_DIR, DB_DIR = '/projekti/mondoAPI', 'db'

types = json.load(open(os.path.join(BASE_DIR, DB_DIR, 'types.json'), 'r'))
types_a = np.array([tuple([b if b else '' for b in a]) if len(a) == 4 else ('', '', '', '') for a in types],
                   [('tip', 'U3'), ('lang', 'U2'), ('script', 'U4'), ('cc', 'U2')])
t2i = {'_'.join(v):i for i, v in enumerate(types_a)}



In [ ]:
i = 0
names = defaultdict(Counter)
nema = Counter()
for l in tqdm(open('/backup/wikidata/fnln.txt'), total=26_332_292):
    try:
        qid, n, lang, fn, ln, tip, gender = l.strip('\n').split('\t')
    except Exception as e:
        print(e, l)
    if fn and ln:
        for prov in get_provenance(n, lang):
            tag_t = 'fn' + ('1' if gender == 'm' else ('2' if gender == 'f' else '0'))
            tag = tag_t +'_'+prov
            if tag in t2i:
                names[fn]['PER_'+tag] += 1
            else:
                names[fn]['PER_'+tag_t+'___'] += 1
                nema[tag] += 1
            tag = 'ln' + '_' + prov
            if tag in t2i:
                names[ln]['PER_'+tag] += 1
            else:
                names[ln]['PER_'+tag_t+'___'] += 1
                nema[tag] += 1
        
names['davor'], names['prof.'], names['prof']

  0%|          | 0/26332292 [00:00<?, ?it/s]

In [ ]:
TODO = defaultdict(set)
for l in tqdm(open('data/wikiid.tsv'), total=834281):
    l = l.strip('\n')
    tip, qid = l.strip('\n').split('\t')
    if qid[0] == 'Q':
        if tip in ('TITLE', 'RANK', 'POS',):
            tip = 'tit'
        else:
            tip = tip.lower()

        qid = int(qid[1:])
        TODO[qid].add(tip)
TODO[121594], len(TODO)

In [ ]:
for qid, tips in tqdm(TODO.items(), total=len(TODO)):
    lab = qid_lab_get(qid, include_alt=True)
    for l, langs in lab.items():
        for lang in langs:
            for tip in tips:
                if tip == 'tit':
                    provs = set([a[:-2] for a in get_provenance(l, lang)])
                else:
                    provs = set([a for a in get_provenance(l, lang)])
                for prov in provs:
                    tag = f'{tip}_'+prov
                    print(tag, tag in t2i)
                    if tag in t2i:
                        names[l][f'W_{tag}'] += 1
                    else:
                        names[l][f'W_{tip}__unkn_'] += 1
                        nema[tag] += 1

names['prof'], names['davor']

In [ ]:
names['davor']